In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("241212_01_MLlib_regression").getOrCreate()

24/12/13 10:21:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


# data load

In [3]:
train_df = spark.read.format("csv")\
    .option("header", 'true')\
    .option('inferSchema', 'true')\
    .load('data/house_train.csv')

In [4]:
test_df = spark.read.format("csv")\
    .option("header", 'true')\
    .option('inferSchema', 'true')\
    .load('data/house_test.csv')

In [5]:
# sale price : label
# features : ???

In [6]:
train_df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- MSSubClass: integer (nullable = true)
 |-- MSZoning: string (nullable = true)
 |-- LotFrontage: string (nullable = true)
 |-- LotArea: integer (nullable = true)
 |-- Street: string (nullable = true)
 |-- Alley: string (nullable = true)
 |-- LotShape: string (nullable = true)
 |-- LandContour: string (nullable = true)
 |-- Utilities: string (nullable = true)
 |-- LotConfig: string (nullable = true)
 |-- LandSlope: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Condition1: string (nullable = true)
 |-- Condition2: string (nullable = true)
 |-- BldgType: string (nullable = true)
 |-- HouseStyle: string (nullable = true)
 |-- OverallQual: integer (nullable = true)
 |-- OverallCond: integer (nullable = true)
 |-- YearBuilt: integer (nullable = true)
 |-- YearRemodAdd: integer (nullable = true)
 |-- RoofStyle: string (nullable = true)
 |-- RoofMatl: string (nullable = true)
 |-- Exterior1st: string (nullable = true)
 |--

In [7]:
test_df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- MSSubClass: integer (nullable = true)
 |-- MSZoning: string (nullable = true)
 |-- LotFrontage: string (nullable = true)
 |-- LotArea: integer (nullable = true)
 |-- Street: string (nullable = true)
 |-- Alley: string (nullable = true)
 |-- LotShape: string (nullable = true)
 |-- LandContour: string (nullable = true)
 |-- Utilities: string (nullable = true)
 |-- LotConfig: string (nullable = true)
 |-- LandSlope: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Condition1: string (nullable = true)
 |-- Condition2: string (nullable = true)
 |-- BldgType: string (nullable = true)
 |-- HouseStyle: string (nullable = true)
 |-- OverallQual: integer (nullable = true)
 |-- OverallCond: integer (nullable = true)
 |-- YearBuilt: integer (nullable = true)
 |-- YearRemodAdd: integer (nullable = true)
 |-- RoofStyle: string (nullable = true)
 |-- RoofMatl: string (nullable = true)
 |-- Exterior1st: string (nullable = true)
 |--

# 전처리
- 결측치 = 0
- 타입 변환
- ~~숫자 타입 통일 (이미 되어있으므로 skip)~~

In [51]:
# 결측치 처리
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)

In [52]:
# 타입 변환 : GarageArea, GarageCars integer로 캐스트
train_df = train_df.withColumn( "GarageArea", train_df["GarageArea"].cast("integer") )
test_df = test_df.withColumn( "GarageArea",  test_df["GarageArea"].cast("integer") )

In [53]:
train_df = train_df.withColumn( "GarageCars", train_df["GarageCars"].cast("integer") )
test_df = test_df.withColumn( "GarageCars",  test_df["GarageCars"].cast("integer") )

In [54]:
train_df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- MSSubClass: integer (nullable = true)
 |-- MSZoning: string (nullable = true)
 |-- LotFrontage: string (nullable = true)
 |-- LotArea: integer (nullable = true)
 |-- Street: string (nullable = true)
 |-- Alley: string (nullable = true)
 |-- LotShape: string (nullable = true)
 |-- LandContour: string (nullable = true)
 |-- Utilities: string (nullable = true)
 |-- LotConfig: string (nullable = true)
 |-- LandSlope: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Condition1: string (nullable = true)
 |-- Condition2: string (nullable = true)
 |-- BldgType: string (nullable = true)
 |-- HouseStyle: string (nullable = true)
 |-- OverallQual: integer (nullable = true)
 |-- OverallCond: integer (nullable = true)
 |-- YearBuilt: integer (nullable = true)
 |-- YearRemodAdd: integer (nullable = true)
 |-- RoofStyle: string (nullable = true)
 |-- RoofMatl: string (nullable = true)
 |-- Exterior1st: string (nullable = true)
 |--

# 인코딩
* 문자형 > 숫자형 (1, 2, 3, 4)

In [8]:
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder

In [55]:
string_columns = ['Neighborhood']
# 1,2,3,4 로 값을 단순화
indexers = [StringIndexer(inputCol=col, outputCol=col+"_index")   for col in string_columns]

In [56]:
# onehotencoding : 범주형변수 1,2,3,4, -> 1로 바꾸는 인코딩 (모델에 미치는 영향을 동일하게 세팅)
# 모두 1로 바꾼다.
encoders = [OneHotEncoder(inputCol=col+"_index", outputCol=col+"_encoded")   for col in string_columns]

# features selection

In [57]:
numeric_columns = ["LotArea", "OverallQual", "OverallCond", "YearBuilt", "YearRemodAdd", 
    "1stFlrSF", "2ndFlrSF", "GrLivArea", "GarageCars", "GarageArea"]

In [58]:
# 문자형(인코딩) + 숫자형 피처를 결합한 모델 입력 생성
assembler_inputs = [col+"_encoded" for col in string_columns] + numeric_columns

# assembler

In [59]:
assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="features")

# label selection

In [60]:
train_df = train_df.withColumnRenamed("SalePrice", "label")

# pipline 설정

In [61]:
from pyspark.ml import Pipeline

In [62]:
pipeline = Pipeline(stages = indexers+encoders+[assembler])

In [63]:
# pipeline 실행
pipeline_model = pipeline.fit( train_df )

In [64]:
train_transformed = pipeline_model.transform( train_df )

# 예측 > 회귀  모델 학습 > 평가 > 예측

In [65]:
from pyspark.ml.regression import LinearRegression

In [66]:
lr = LinearRegression(featuresCol="features", labelCol="label")
lr_model = lr.fit(train_transformed)

24/12/13 10:54:42 WARN Instrumentation: [ff3c4666] regParam is zero, which might cause numerical instability and overfitting.


In [67]:
# 평가 데이터 를 이용한 평가> FIT - 모델 맞춤 과정, 테스트 데이터에 의해 규칙이 변화

test_transformed = pipeline_model.transform( test_df )
predictions = lr_model.transform(test_transformed)

In [68]:
# 예측값 확인

predictions.select("id", "features", "prediction").show(10, truncate=False)

+----+-------------------------------------------------------------------------------------------------------+------------------+
|id  |features                                                                                               |prediction        |
+----+-------------------------------------------------------------------------------------------------------+------------------+
|1461|(34,[0,24,25,26,27,28,29,31,32,33],[1.0,11622.0,5.0,6.0,1961.0,1961.0,896.0,896.0,1.0,730.0])          |114113.60325331613|
|1462|(34,[0,24,25,26,27,28,29,31,32,33],[1.0,14267.0,6.0,6.0,1958.0,1958.0,1329.0,1329.0,1.0,312.0])        |156145.8445868329 |
|1463|(34,[5,24,25,26,27,28,29,30,31,32,33],[1.0,13830.0,5.0,5.0,1997.0,1998.0,928.0,701.0,1629.0,2.0,482.0])|168254.6697326172 |
|1464|(34,[5,24,25,26,27,28,29,30,31,32,33],[1.0,9978.0,6.0,6.0,1998.0,1998.0,926.0,678.0,1604.0,2.0,470.0]) |186898.44701529457|
|1465|(34,[18,24,25,26,27,28,29,31,32,33],[1.0,5005.0,8.0,5.0,1992.0,1992.0,1280.0,1280.0,

In [69]:
# 예측 결과 저장
predictions.select("id", "prediction") \
.withColumnRenamed('prediction', 'SalePrice') \
.write.csv('data/output/house_prediction.csv', header=True, mode='overwrite')

In [ ]:
# 예측값 읽어서 분석 (차트 등)

# 예측 모델의 활용
1. 파이프라인 저장 > 로컬 data/output/ > 모델 저장소에 저장
2. 모델 저장 > 로컬 > 모델저장소에 저장

In [70]:
model_save_path = 'data/output/boston_housing_lr_model'
pipeline_save_path = 'data/output/boston_housing_pipeline_model'

#파이프라인모델 > 새로운 데이터를 변환하기 위해 저장
pipeline_model.write().overwrite().save(pipeline_save_path)

#선형회귀모델 > 새로운 데이터로 예측하기 위해 저장
lr_model.write().overwrite().save(model_save_path)
print('model saved..')

model saved..


# 모델, 파이프라인 로드

In [73]:
from pyspark.ml import PipelineModel
from pyspark.ml.regression import LinearRegressionModel

In [74]:
loaded_pipeline = PipelineModel.load(pipeline_save_path)
loaded_pipeline

PipelineModel_eb5d43ce9ee5

In [75]:
loaded_model = LinearRegressionModel.load(model_save_path)
loaded_model

LinearRegressionModel: uid=LinearRegression_82069a45bee8, numFeatures=34

# 새로운 데이터로 예측

1. 새로운 데이터 >>> ???
2. 파이프라인모델을 이용해서 변환
3. 모델에 넣어 예측

In [76]:
import pandas as pd
# 새로운 데이터 샘플 생성
data = {
    "Id": [1461],
    "MSSubClass": [20],
    "MSZoning": ["RH"],
    "LotFrontage": [80],
    "LotArea": [11622],
    "Street": ["Pave"],
    "Alley": [None],  # NA를 None으로 표현
    "LotShape": ["Reg"],
    "LandContour": ["Lvl"],
    "Utilities": ["AllPub"],
    "LotConfig": ["Inside"],
    "LandSlope": ["Gtl"],
    "Neighborhood": ["NAmes"],
    "Condition1": ["Feedr"],
    "Condition2": ["Norm"],
    "BldgType": ["1Fam"],
    "HouseStyle": ["1Story"],
    "OverallQual": [5],
    "OverallCond": [6],
    "YearBuilt": [1961],
    "YearRemodAdd": [1961],
    "RoofStyle": ["Gable"],
    "RoofMatl": ["CompShg"],
    "Exterior1st": ["VinylSd"],
    "Exterior2nd": ["VinylSd"],
    "MasVnrType": [None],  # None은 NA를 의미
    "MasVnrArea": [0],
    "ExterQual": ["TA"],
    "ExterCond": ["TA"],
    "Foundation": ["CBlock"],
    "BsmtQual": ["TA"],
    "BsmtCond": ["TA"],
    "BsmtExposure": ["No"],
    "BsmtFinType1": ["Rec"],
    "BsmtFinSF1": [468],
    "BsmtFinType2": ["LwQ"],
    "BsmtFinSF2": [144],
    "BsmtUnfSF": [270],
    "TotalBsmtSF": [882],
    "Heating": ["GasA"],
    "HeatingQC": ["TA"],
    "CentralAir": ["Y"],
    "Electrical": ["SBrkr"],
    "1stFlrSF": [896],
    "2ndFlrSF": [0],
    "LowQualFinSF": [0],
    "GrLivArea": [896],
    "BsmtFullBath": [0],
    "BsmtHalfBath": [0],
    "FullBath": [1],
    "HalfBath": [0],
    "BedroomAbvGr": [2],
    "KitchenAbvGr": [1],
    "KitchenQual": ["TA"],
    "TotRmsAbvGrd": [5],
    "Functional": ["Typ"],
    "Fireplaces": [0],
    "FireplaceQu": [None],  # NA를 None으로 표현
    "GarageType": ["Attchd"],
    "GarageYrBlt": [1961],
    "GarageFinish": ["Unf"],
    "GarageCars": [1],
    "GarageArea": [730],
    "GarageQual": ["TA"],
    "GarageCond": ["TA"],
    "PavedDrive": ["Y"],
    "WoodDeckSF": [140],
    "OpenPorchSF": [0],
    "EnclosedPorch": [0],
    "3SsnPorch": [0],
    "ScreenPorch": [120],
    "PoolArea": [0],
    "PoolQC": [None],  # NA를 None으로 표현
    "Fence": ["MnPrv"],
    "MiscFeature": [None],  # NA를 None으로 표현
    "MiscVal": [0],
    "MoSold": [6],
    "YrSold": [2010],
    "SaleType": ["WD"],
    "SaleCondition":["Normal"]
}
pd.DataFrame(data).to_csv('data/new_test_data.csv', index=False)

In [77]:
new_test_data = spark.read.csv("data/new_test_data.csv", header=True, inferSchema=True)

In [80]:
# 필요한 특성만 선택 (파이프라인에서 사용된 특성들)
# 수치형 컬럼 + 범주형 컬럼 정의
selected_features = [
    "LotArea", "OverallQual", "OverallCond", "YearBuilt", "YearRemodAdd", 
    "1stFlrSF", "2ndFlrSF", "GrLivArea", "GarageCars", "GarageArea", "Neighborhood"
]

In [81]:
# 데이터 타입 변환 및 필요한 특성 선택
new_test_data = new_test_data.withColumn("GarageCars", new_test_data["GarageCars"].cast("integer"))
new_test_data = new_test_data.withColumn("GarageArea", new_test_data["GarageArea"].cast("integer"))

In [86]:
# 파이프라인이 변환한 데이터를 모델에 넣어 준다.
new_pipe_data = loaded_pipeline.transform(new_test_data)

In [87]:
# 파이프라인이 변환한 데이터를 모델에 넣어 준다.
new_pred = loaded_model.transform( new_pipe_data ) #1건

In [88]:
# 예측 수행
new_pred.select("prediction").show()

+------------------+
|        prediction|
+------------------+
|114113.60325331613|
+------------------+



In [89]:
# CSV 저장

In [91]:
# LOG를 남긴다. - CSV 저장, 데이터베이스에 저장 "외부저장", 하둡 분산파일 시스템 HDFS, kafka(Streaming)

In [92]:
spark.stop()

➕ 요약
### 데이터셋 구성

- **train_df**와 **test_df** 데이터를 CSV 파일에서 불러왔음.
- 결측치를 처리하고, **GarageArea**와 **GarageCars**의 타입을 변환했음.

### 모델 및 파이프라인 준비

- 문자열 데이터를 숫자로 변환하기 위해 **StringIndexer**와 **OneHotEncoder**를 사용했음.
- **VectorAssembler**를 사용해 여러 피처를 하나의 벡터로 결합했음.
- 파이프라인을 설정하고 데이터를 학습시켰음.

### 모델 학습

- **train_transformed** 데이터를 사용해 선형 회귀 모델을 학습시켰음.

### 모델 예측

- 학습된 모델을 사용해 **test_df** 데이터를 예측했음.

### 모델 테스트 및 정확도 평가
모델의 성능을 평가하기 위해서는 예측값과 실제 값을 비교해 정확도를 측정해야 함.  
일반적으로 회귀 모델의 성능을 평가할 때는 다음과 같은 지표를 사용함:

- **Mean Absolute Error (MAE)**: 예측값과 실제 값의 차이의 절대값의 평균
- **Mean Squared Error (MSE)**: 예측값과 실제 값의 차이의 제곱의 평균
- **Root Mean Squared Error (RMSE)**: MSE의 제곱근
- **R-squared (R²)**: 모델이 데이터를 얼마나 잘 설명하는지 나타내는 지표

#### 예시 코드 (모델 성능 평가)
```python
from pyspark.ml.evaluation import RegressionEvaluator

# 평가기 설정
evaluator = RegressionEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="rmse"
)

# 모델 평가
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
